## Library Import

In [38]:
import duckdb
import polars as p

import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from config import load_setting
settings = load_setting()

## Load Silver Data

In [39]:
# Test connection to Silver Data db
try:
    db_path = settings['process']['silver_db_path']
    print(f"Attempting to connect to: {db_path}")
    
    # Try connection
    con = duckdb.connect(f"../{db_path}")
    
    # Test Connection with a simple query
    result = con.execute("SELECT 1").fetchone()
    print("Connection Succesfful!")
    print(f"Test query result: {result}")
    
    
    # List all tables in the database
    tables = con.execute("SHOW TABLES").fetchall()
    print("\nAvailable tables:", tables)
    
except Exception as e:
    print(f"Connection Failed {str(e)}")

Attempting to connect to: process/storage/silver/resampled.duckdb
Connection Succesfful!
Test query result: (1,)

Available tables: [('gold_data',), ('raw_data',), ('silver_1',), ('silver_13',), ('silver_21',), ('silver_3',), ('silver_34',), ('silver_5',), ('silver_55',), ('silver_8',)]


In [40]:
# List all tables
tables = con.execute("SHOW TABLES").fetchall()
print("Available tables:", tables)

Available tables: [('gold_data',), ('raw_data',), ('silver_1',), ('silver_13',), ('silver_21',), ('silver_3',), ('silver_34',), ('silver_5',), ('silver_55',), ('silver_8',)]


In [41]:
# Get schema information for a specific table
# Replace 'your_table_name' with an actual table name
table_info = con.execute("DESCRIBE silver_1").fetchall()
print("Table schema:", table_info)

Table schema: [('symbol', 'VARCHAR', 'YES', None, None, None), ('date', 'TIMESTAMP WITH TIME ZONE', 'YES', None, None, None), ('open', 'DOUBLE', 'YES', None, None, None), ('high', 'DOUBLE', 'YES', None, None, None), ('low', 'DOUBLE', 'YES', None, None, None), ('close', 'DOUBLE', 'YES', None, None, None), ('volume', 'DOUBLE', 'YES', None, None, None)]


In [42]:
# Get sample data from the table
sample_data = con.execute("SELECT * FROM silver_1 LIMIT 5").fetchall()
print("Sample data:", sample_data)

Sample data: [('VRNS', datetime.datetime(2019, 1, 25, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 18.0833320618, 18.4699993134, 17.9133338928, 18.3733329773, 476700.0), ('VRNS', datetime.datetime(2019, 3, 5, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 19.6033325195, 20.2233333588, 19.4233322144, 20.0766677856, 1911600.0), ('VRNS', datetime.datetime(2019, 8, 8, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 22.8866672516, 23.703332901, 22.8866672516, 23.3566665649, 895200.0), ('VRNS', datetime.datetime(2019, 10, 29, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 23.4333324432, 24.6996669769, 22.3366661072, 23.8233337402, 4088700.0), ('VRNS', datetime.datetime(2019, 11, 12, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 24.3366661072, 24.7833328247, 24.2466678619, 24.6700000763, 741900.0)]


In [43]:
# Get the date range of the silver_1 table
con.execute("SELECT MIN(date), MAX(date) FROM silver_1").fetchall() 

[(datetime.datetime(1962, 1, 2, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>),
  datetime.datetime(2025, 6, 11, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>))]

In [44]:
# Get the first 5 rows of Resampled Data
first_5_rows = con.execute("SELECT * FROM silver_1 ORDER BY date DESC LIMIT 5").fetchall()
print("First 5 rows:", first_5_rows)

First 5 rows: [('PRLD', datetime.datetime(2025, 6, 11, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 1.11, 1.16, 1.06, 1.1, 206029.0), ('HDB', datetime.datetime(2025, 6, 11, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 76.38, 76.79, 75.89, 76.025, 1264502.0), ('SEM', datetime.datetime(2025, 6, 11, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 15.39, 15.62, 15.07, 15.08, 830062.0), ('NUWE', datetime.datetime(2025, 6, 11, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 0.24, 0.5857, 0.2346, 0.3731, 431005861.0), ('NRG', datetime.datetime(2025, 6, 11, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 150.51, 152.0, 148.31, 148.995, 3031465.0)]


In [45]:
# Get total row count
row_count = con.execute("SELECT COUNT(*) FROM silver_1").fetchone()
print("Total rows:", row_count)

Total rows: (6182893,)


In [46]:
# Get the first 5 rows of the table
first_5_rows = con.execute("SELECT * FROM silver_1 LIMIT 5").fetchall()
print("First 5 rows:", first_5_rows)
# Get the last 5 rows of the table
last_5_rows = con.execute("SELECT * FROM silver_1 ORDER BY date DESC LIMIT 5").fetchall()
print("Last 5 rows:", last_5_rows)


First 5 rows: [('VRNS', datetime.datetime(2019, 1, 25, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 18.0833320618, 18.4699993134, 17.9133338928, 18.3733329773, 476700.0), ('VRNS', datetime.datetime(2019, 3, 5, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 19.6033325195, 20.2233333588, 19.4233322144, 20.0766677856, 1911600.0), ('VRNS', datetime.datetime(2019, 8, 8, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 22.8866672516, 23.703332901, 22.8866672516, 23.3566665649, 895200.0), ('VRNS', datetime.datetime(2019, 10, 29, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 23.4333324432, 24.6996669769, 22.3366661072, 23.8233337402, 4088700.0), ('VRNS', datetime.datetime(2019, 11, 12, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 24.3366661072, 24.7833328247, 24.2466678619, 24.6700000763, 741900.0)]
Last 5 rows: [('PRLD', datetime.datetime(2025, 6, 11, 0, 0, tzinfo=<DstTzInfo '

In [47]:
# Get column statistics
stats = con.execute("""
    SELECT 
        symbol,
        COUNT(*) as count,
        AVG(open) as avg,
        MIN(open) as min,
        MAX(open) as max
    FROM silver_1
    GROUP BY symbol
""").fetchall()
print("Column statistics:", stats)

Column statistics: [('MFH', 2561, 28.330560639202893, 0.6144440174, 286.6666564941), ('NXTT', 728, 3.735546572319093, 0.0410000011, 49.0), ('SBXD', 170, 10.167277089028822, 9.9499998093, 10.84), ('VRNS', 2839, 26.57107338332607, 4.5133328438, 73.2900009155), ('DORM', 8626, 30.751942144341324, 0.373734313, 143.3099975586), ('VIK', 279, 39.52548741904659, 26.1499996185, 53.0800018311), ('DSGX', 6635, 25.316351181813687, 0.8000000119, 122.5199966431), ('MDB', 1921, 237.47672341981624, 25.6200008392, 585.0300292969), ('CLPT', 3283, 19.607697214073955, 1.3999999762, 182.3999938965), ('DPZ', 5264, 155.74649498746996, 2.3579237456, 539.9996756702), ('ALLY', 2861, 25.237922966111046, 9.7879806083, 49.2568587757), ('HITI', 1111, 3.2804941497969398, 1.0599999428, 12.75), ('AORT', 8138, 12.26395682447156, 1.4302930423, 41.8599018269), ('BKNG', 6591, 1049.1327501852356, 6.5286219486, 5615.56), ('JRSH', 1786, 4.393678794088073, 2.5889962752, 7.6104504813), ('NSA', 2551, 27.435848227638648, 7.468143

In [48]:
import polars as pl

# Turn stats into a pl dataframe with mixed types allowed
stats_df = pl.DataFrame(stats, 
                        schema={
                            "symbol": pl.Utf8,
                            "count": pl.Int64,
                            "avg": pl.Float64,
                            "min": pl.Float64,
                            "max": pl.Float64
                        })
print(stats_df)

shape: (1_458, 5)
┌────────┬───────┬────────────┬──────────┬────────────┐
│ symbol ┆ count ┆ avg        ┆ min      ┆ max        │
│ ---    ┆ ---   ┆ ---        ┆ ---      ┆ ---        │
│ str    ┆ i64   ┆ f64        ┆ f64      ┆ f64        │
╞════════╪═══════╪════════════╪══════════╪════════════╡
│ MFH    ┆ 2561  ┆ 28.330561  ┆ 0.614444 ┆ 286.666656 │
│ NXTT   ┆ 728   ┆ 3.735547   ┆ 0.041    ┆ 49.0       │
│ SBXD   ┆ 170   ┆ 10.167277  ┆ 9.95     ┆ 10.84      │
│ VRNS   ┆ 2839  ┆ 26.571073  ┆ 4.513333 ┆ 73.290001  │
│ DORM   ┆ 8626  ┆ 30.751942  ┆ 0.373734 ┆ 143.309998 │
│ …      ┆ …     ┆ …          ┆ …        ┆ …          │
│ DTSS   ┆ 1829  ┆ 49.687124  ┆ 1.43     ┆ 278.100006 │
│ HLN    ┆ 724   ┆ 8.364819   ┆ 5.484787 ┆ 11.35      │
│ ABP    ┆ 818   ┆ 8.863107   ┆ 0.1733   ┆ 12.4       │
│ BILL   ┆ 1381  ┆ 114.096662 ┆ 27.5     ┆ 345.829987 │
│ PLUT   ┆ 88    ┆ 2.97565    ┆ 2.22     ┆ 4.17       │
└────────┴───────┴────────────┴──────────┴────────────┘


In [49]:
con.close()
